Text Classification Using FastText <br>
Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification

We have a dataset of ecommerce item description. Total 4 categories,

    Household
    Electronics
    Clothing and Accessories
    Books

The task at hand is to classify a product into one of the above 4 categories based on the product description


In [1]:
import pandas as pd

df= pd.read_csv("ecommerce_dataset.csv", names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [2]:
# Drop NA Values

df.dropna(inplace=True)
df.shape

(50424, 2)

In [3]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [4]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8670
Name: count, dtype: int64

In [5]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

/tmp/ipykernel_9267/2912538940.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)


In [7]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

When you train a fasttext model, it expects labels to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the product description

In [8]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [9]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


In [ ]:
#df['category_description'][0]



Pre-procesing

    Remove punctuation
    Remove extra space
    Make the entire sentence lower case



In [10]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"   # Sample sentence
text = re.sub(r'[^\w\s\']',' ', text)    # sub means subtitute
text = re.sub(' +', ' ', text)
text.strip().lower()

"viki's bookcase bookshelf 3 shelf shelve white hi"

In [11]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [12]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()


,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


Train Test Split

In [13]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [14]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [15]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)




Train the model and evaluate performance


In [16]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

Read 4M words
Number of words:  79352
Number of labels: 4
Progress: 100.0% words/sec/thread: 2684337 lr:  0.000000 avg.loss:  0.185136 ETA:   0h 0m 0s


(10084, 0.9691590638635462, 0.9691590638635462)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good



Now let's do prediction for few product descriptions


In [17]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.98314363]))

In [18]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [19]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000978]))

In [20]:
model.get_nearest_neighbors("painting")

[(0.998742401599884, 'wigs'),
 (0.9987353086471558, 'cm\xa0package'),
 (0.9987314939498901, 'musthaves'),
 (0.9987257719039917, 'pricesome'),
 (0.9987257719039917, 'friendsthis'),
 (0.9987218976020813, 'padlocks'),
 (0.9987216591835022, 'inchfeatures'),
 (0.9987216591835022, 'earsoccasions'),
 (0.9987216591835022, 'materialcolor'),
 (0.9987216591835022, 'colorsize')]

In [21]:
model.get_nearest_neighbors("sony")

[(0.9994998574256897, '170d'),
 (0.9994997978210449, 'wch'),
 (0.9994997978210449, 'ch382l'),
 (0.9994850754737854, "acoustics'"),
 (0.9994834661483765, '530mb'),
 (0.9994823336601257, 'gapless'),
 (0.999465823173523, 'mu06'),
 (0.9994524121284485, 'dcd'),
 (0.9994485974311829, 'a320m'),
 (0.9994432926177979, 'whereabouts')]

In [22]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, '400hours'),
 (0.0, '146x40x19mm'),
 (0.0, '6x0')]